## Introduction

The goal of this lab is to construct a simple "linear regression" network to detect bird call activities in an audio file. Via this lab, you will learn about processing audio files, building a network with Pytorch, training the network with labeled data and evaluating it, and data visualization. 

In [ ]:
# Import the packages we'll use

import numpy as np
import os, glob, csv

# librosa is a widely-used audio processing library
import librosa

#sklearn is a useful toolkit for machine learning
import sklearn

#pytorch: deep learning framework
import torch
import torch.nn as nn
import torch.nn.functional as nnF

# for plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# USER CONFIGURATION
# Please alter the paths here to where the data are stored on your local filesystem

#binarylabelcsv  = os.path.expanduser("~/datasets/badch2/badch2metadata/developmentsets/warblrb10k_public_metadata_2018.csv")
#binarylabelcsv  = os.path.expanduser("~/git/stored_docs/fship2014e/badchallenge/badch2metadata/developmentsets/warblrb10k_public_metadata_2018.csv")
#binarylabelcsv  = "/import/teaching/DATASETS/ECS7013P/bird_audio_detection/warblrb10k_public_metadata_2018.csv"
binarylabelcsv  = ""

#audiofilefolder = os.path.expanduser("~/datasets/badch2/devel_wav/warblrb10k/wav")
#audiofilefolder = os.path.expanduser("~/audio_data/warblrb10k/wav")
#audiofilefolder = "/import/teaching/DATASETS/ECS7013P/bird_audio_detection/warblrb10k/wav"
audiofilefolder = ""

maxfilestoload  = 50      # limit, because loading the whole dataset is very slow


In [ ]:
# here we load the metadata labels
binarylabels = {}
with open(binarylabelcsv, 'r') as infp:
        rdr = csv.DictReader(infp)
        for row in rdr:
            
                # view structure of the "warblrb10k_public_metadata_2018.csv" file
                # complete the line below to read the map the label (hasbird) to the corresponding audio file name
                # binarylabels[row[...]] = float(row[...])
                
                if len(binarylabels)==maxfilestoload:
                        break  # note, here we are restricting the maximum number of rows.

fkeys = sorted(binarylabels.keys())
# inspect:
for i, kv in enumerate(binarylabels.items()):
    print(kv)
    if i==10: break

In [ ]:
# load an example audio file, converting the data to mel spectrogram
examplefkey = '1d94fc4a-1c63-4da0-9cac'

example_audio, sr = librosa.load("%s/%s.wav" % (audiofilefolder, examplefkey), sr=22050) # note the sampling frequency
example = librosa.feature.melspectrogram(example_audio, sr, n_mels=64) # note the number of mel bands

# player
import IPython.display as ipd
audio_element_url = ipd.Audio(example_audio, rate=sr)
ipd.display(audio_element_url)


# Let's look at the spectrogram of one example
plt.imshow(librosa.power_to_db(example, ref=np.max), aspect='auto', origin='lower')
plt.colorbar()

In [ ]:
print(example.shape)

Let's annotate this audio clip in detail, specifying for each *frame* whether it has any bird call active or not.

In [ ]:
annotation = np.zeros(434)
for onset, offset in [
    (44, 64),
    (120, 140),
    (170, 188),
    (203, 218),
    (254, 320),
    (338, 359),
    (368, 390),
    (400, 414),
                     ]:
    annotation[onset:offset] = 1
    
plt.imshow(librosa.power_to_db(example, ref=np.max), aspect='auto', origin='lower')
plt.plot(annotation*15)

Next we want to establish if there is any *linear* relationship between the energy in the frequency bands, and the regressor (the "independent variable") which is our annotation.

In [ ]:
# As a simple example, here's a scatter plot of the energy in frequency band index "31" and our annotation.
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[1].plot(example[31,:], annotation, 'x')
ax[1].set_xlabel("Energy in band 31")
ax[1].set_ylabel("Bird activity")

ax[0].plot(annotation * 50, label="True")
ax[0].plot(example[31,:], label="Band 31 energy")
ax[0].set_xlabel("Time")
ax[0].set_ylabel("Bird activity")
ax[0].legend()


print("Mean energy when activity is 0: %g" % np.mean(example[31, np.nonzero(annotation==0)]))
print("Mean energy when activity is 1: %g" % np.mean(example[31, np.nonzero(annotation==1)]))

In [ ]:
# Let's set this problem up within the Torch framework:
def sigmoid(x):
    "Applies a sigmoid nonlinearity to a numpy vector of values"
    # complete this function
    sm = ...
    return sm

class LinearRegress(nn.Module):
    def __init__(self, nfreqs):
        super(LinearRegress, self).__init__()
        # initialize the parameters with random values
        self.beta = nn.Parameter(torch.randn((nfreqs,1)).abs() * 1e-4)
        self.offset = nn.Parameter(torch.rand((nfreqs,1)) * 1e-4)
        # question: what is the difference between "torch.randn" and "torch.rand"?

    def forward(self, x):
        "Project the data through the coefficients"
        # complete this function
        y = ...
        return y
    
    def forward_and_convert(self, x):
        "Handles the torch<--->numpy tensor conversion, for convenience"
        x_torch = torch.DoubleTensor(x)
        y_torch = self.forward(x_torch)
        return y_torch.detach().numpy()

In [ ]:
# create a LinearRegress network instance
net = LinearRegress(64)
# our network is currently RANDOMLY initialised. What happens when we project our audio through it?
plt.plot(annotation, label="True")
plt.plot(net.forward_and_convert(example), label="Predicted")
plt.xlabel("Time")
plt.ylabel("Bird activity")
plt.legend()
plt.title("Linear regression as bird detector - BEFORE fitting")

In [ ]:
# define a loss function and an optimiser
# which loss function should we use for this problem, compelte the line below
criterion = ...   
optimizer = torch.optim.SGD(net.parameters(), lr=0.0000001, momentum=0.8) # optimizer

In [ ]:
#annotation

In [ ]:
# train the network
num_epochs = 10000 # number of training epochs
annot_torch = torch.DoubleTensor(annotation)
for epoch in range(num_epochs):

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    # complete the lines below
    outputs = ... # forward pass
    loss = ... # loss
    ... # backward pass
    
    optimizer.step()

    running_loss = loss.item()
    if ((epoch & (epoch - 1)) == 0) or epoch==(num_epochs-1): # don't print on all epochs
        # print statistics
        print('[%d] loss: %.8f' %
            (epoch, running_loss))

example_result = outputs.detach().numpy()
print('Finished Training')

In [ ]:
plt.plot(annotation, label="True")
plt.plot(net.forward_and_convert(example), label="Predicted")
plt.xlabel("Time")
plt.ylabel("Bird activity")
plt.legend()
plt.title("Linear regression as bird detector")

In [ ]:
### Let's compare the linear regression predictions against our "energy in band 31" scatter
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].plot(annotation, label="True")
ax[1].plot(example[31,:], annotation, 'x', label="True")

### This code makes a prediction with band 31 only
### (replacing other bands with their average energy, to keep them fixed)
if False:
    # Question: can you explain what the line below does?
    example_reduced = np.repeat(np.mean(example, axis=1, keepdims=True), repeats=example.shape[1], axis=1)
    example_reduced[31,:] = example[31,:]
    ax[0].plot(net.forward_and_convert(example_reduced), label="Predicted (31 only)")
    ax[0].set_xlabel("Time")
    ax[0].set_ylabel("Bird activity")
    ax[0].legend()
    ax[1].plot(example[31,:], net.forward_and_convert(example_reduced), '.', label="Predicted (31 only)")

### This plots the "full" regression prediction from all data
if False:
    ax[0].plot(net.forward_and_convert(example), label="Predicted (full)")
    ax[0].set_xlabel("Time")
    ax[0].set_ylabel("Bird activity")
    ax[0].legend()
    ax[1].plot(example[31,:], net.forward_and_convert(example), '.', label="Predicted (full)")

ax[1].set_xlabel("Energy in band 31")
ax[1].set_ylabel("Bird activity")
ax[1].legend()

In [ ]:
# Let's inspect the parameters
plt.figure()
plt.plot(net.offset.detach().numpy())
plt.title("Offsets")
plt.xlabel("Frequency")
plt.figure()
plt.plot(net.beta.detach().numpy())
plt.title("Beta coefficients")
plt.xlabel("Frequency")

Exercise
-------

* Convert this from a linear regression to a *logistic* regression. Retrain it and inspect how well it can do.
* Experiment with the optimiser options such as learning rate and momentum.